In [1]:
import os
import pymongo
import sys
import json
import time
import urllib.parse
import pandas as pd
from datetime import timedelta 
from datetime import datetime
from pymongo import MongoClient

client = MongoClient(port=27017) #Establishing connection with MongoDB
client.admin.authenticate('DasSystem', 'DasSystem',mechanism='SCRAM-SHA-1', source='admin')
db = client["DASMongoDB"] #Database Name
table_name = "DAS.Grob21.Package_2"
basepath = "D://Predictions//MongoData" #Set the required path to save the files

#Initialize required valve nummber and its selected toolId, div_type, percentage of dividing of data
valve_type_no_list = [                         
        {                                   ### div_type = 1 means part A of dataset will be assigned to workpiece 1 and 3
         "number":17010847,                 ### and part B of dataset will be assigned to workpiece 2 and 4
         "selected_toolIds" :[              ### div_type = 2 means part A of dataset will be assigned to workpiece 2 and 4
            {                               ### and part B of dataset will be assigned to workpiece 1 and 3
                "toolId" : 17015,           ### Please add commma(,) before adding new valve type number or new tool
                "div_type" : 1,
                "percentage" :[10,38,4,38,10]
            },
            {
                "toolId" : 17017,
                "div_type" : 2,
                "percentage" :[10,38,4,38,10]
            }
         ] 
        }
]

#add table names which is created in mongoDB e.g. Axis_A table in mongodb created 4 files i.e. Value1_A,Value2_A,Value3_A, Value4_A
allDrive_tables ={
#        "axis_a" : ['Value1_A','Value2_A','Value3_A','Value4_A'],
#        "axis_x" : ['Value1_X','Value2_X','Value3_X','Value4_X'],
#        "axis_y" : ['Value1_Y','Value2_Y','Value3_Y','Value4_Y'],
        "axis_z1" : ["TrendingSp1_Value1","TrendingSP1_Value2","TrendingSp1__Value3","TrendingSp1_Value4"] ,
        "axis_Z2" : ["TrendingSp2_Value1","TrendingSp2_Value2","TrendingSp2__Value3","TrendingSp2_Value4"],
#        "axis_Sp1" : ['Value1_spindle1','Value2_spindle1','Value3_spindle1','Value4_spindle1'],
#        "axis_sp2" : ['Value1_spindle2','Value2_spindle2','Value3_spindle2','Value4_spindle2'],
      
}

#Function to create folders and file for each Tool
def CreateFolder(folder_Name,sub_Fname,fileName,dataframe):
    
    directory= os.path.join(basepath,folder_Name)
    subDirectory= os.path.join(directory,sub_Fname )
    fileName= os.path.join(subDirectory,fileName )
    fileCheck= fileName + ".csv"
    
    #Create new directory if not present
    if not os.path.exists(directory):
        os.makedirs(directory)
        if not os.path.exists(subDirectory): #Create new sub directory if not present 
            os.makedirs(subDirectory)
    elif not os.path.exists(subDirectory): #Create new sub directory inside directory,if directory  exists
        os.makedirs(subDirectory)
    

    #Create new csv file for each table that are defined in above allDrive_tables variable
    try:
        dataframe.to_csv(fileCheck, encoding='utf-8', index=True)
        return fileCheck , sub_Fname ,folder_Name 
    except:
        print("error in creating csv file")
        
#Function to split data into some percentage and creates new folder for tools and files as "toolId_final" and "tablename_final"    
def data_split_types(directory,subDirectory,created_file,wp_no,valve_num):
    for valve_no in valve_type_no_list:
        tool = os.path.basename(subDirectory)
        for selected_toolId in valve_no['selected_toolIds']:
            if valve_no["number"] == valve_num and tool == str(selected_toolId["toolId"]):
                sub = str(subDirectory)+"_final"
                file = os.path.basename(created_file)
                file = file.split('.')
                df = pd.read_csv(created_file)
                total = df['Data'].count()
                
                #Dividing the data into five parts from the original file
                part1 = int(total*selected_toolId["percentage"][0]/100)
                part2 = int(total*selected_toolId["percentage"][1]/100)
                part3 = int(total*selected_toolId["percentage"][2]/100)
                part4 = int(total*selected_toolId["percentage"][3]/100)
                part5 = int(total*selected_toolId["percentage"][4]/100)

                partA = part1 + part2
                partB = partA + part3 + part4 

                #Selecting required part from data file
                selected_partA = df['Data'].iloc[part1:partA]
                selected_partB = df['Data'].iloc[partA+part3:partB]
                selected_partA = pd.DataFrame(selected_partA)
                selected_partA.columns = ['Data']
                selected_partB = pd.DataFrame(selected_partB)
                selected_partB.columns = ['Data']

                if selected_toolId['div_type'] == 1:
                    if str(wp_no) == str(1) or str(wp_no) == str(3):
                        CreateFolder(directory,sub,str(file[0]+"_final"),selected_partA )
                    elif str(wp_no) == str(2)or str(wp_no) == str(4):
                        CreateFolder(directory,sub,str(file[0]+"_final"),selected_partB )
                elif selected_toolId['div_type'] == 2:
                    if str(wp_no) == str(2) or str(wp_no) == str(4):
                        CreateFolder(directory,sub,str(file[0]+"_final"),selected_partA )
                    elif str(wp_no) == str(1)or str(wp_no) == str(3):
                        CreateFolder(directory,sub,str(file[0]+"_final"),selected_partB )

#Called from Automated Process/Automated Process Standalone to start creating the folders                        
def start(begin,end):
    Integrated_folders = []
    for valve_no in valve_type_no_list:
        for selected_toolId in valve_no['selected_toolIds']:
            table = db[table_name].find() #Table name
            for toolId in table:
              if ((toolId["DateTime"] >= begin) and (toolId["DateTime"] <= end)): 
                if toolId["Tool_ID"] == selected_toolId['toolId'] and toolId["Valve_Type_Number"] == valve_no["number"]:
                    workpiece_Ids = []
                    current_Id = toolId['_id']
                    workpiece_Ids.extend([toolId['Workpiece_1'],toolId['Workpiece_2'],toolId['Workpiece_3'],toolId['Workpiece_4']])
                    from_date = ""
                    to_date = ""
                    #Converting time value to string
                    time_string= toolId["DateTime"].strftime("%Y-%m-%d %H:%M:%S.%f")
                    fulldate = datetime.strptime(time_string,"%Y-%m-%d %H:%M:%S.%f")
                    ##Adding 2hr offset to match with system time
                    fulldate = fulldate + timedelta(hours=0) ##Not required now as timestamp synced with NTP Server
                    from_date= fulldate.strftime("%Y-%m-%d %H:%M:%S.%f")
                    cursor = db[table_name].find({"_id": { "$gt": current_Id}}).sort("_id").limit(1)
                    for doc in cursor:
                        time_string= doc["DateTime"].strftime("%Y-%m-%d %H:%M:%S.%f")
                        to_date = datetime.strptime(time_string,"%Y-%m-%d %H:%M:%S.%f")
                        to_date = to_date + timedelta(hours=0)
                        to_date = to_date.strftime("%Y-%m-%d %H:%M:%S.%f")

                        for table in allDrive_tables:
                            for item in allDrive_tables[table]:
                                tab = db[item]
                                #Selecting a item inside Dataframe table with in time range(from_date & to_date)
                                if to_date is not "":
                                    selected_values = tab.find({"DateTime":{"$lte":(to_date),"$gte":(from_date)}})
                                    dataArray = []
                                    for value in selected_values:
                                        for items in value["DataItem"]:
                                            dataArray.append([items["TimeStamp"],items["Data"]])
                                    if len(dataArray) is not 0:
                                        dataset=pd.DataFrame(dataArray)
                                        dataset.columns = ['Timestamp', 'Data']
                                        for wpId in workpiece_Ids:
                                            wp_no = wpId[8:-7] #Splitting workpiece nummber from workpiece Id
                                            folderName= str(valve_no["number"])+'_'+str(wpId)
                                            subFname= str(selected_toolId['toolId'])                           
                                            if folderName not in Integrated_folders:
                                             Integrated_folders.append(folderName); #To store all the folder names            
                                            if table.lower() == "axis_sp1" or table.lower() == "axis_z1":
                                                #Pass spindel1 and Z1 axis table values for workpiece 1 and 2
                                                if str(wp_no) == str(1) or str(wp_no) == str(2):
                                                    created_file,subDirectory, directory = CreateFolder(folderName,subFname,item,dataset)
                                                    data_split_types(directory,subDirectory,created_file,wp_no,valve_no["number"])
                                            elif table.lower() == "axis_sp2" or table.lower() == "axis_z2":
                                                #Pass spindel2 and Z2 axis table values for workpiece 3 and 4
                                                if str(wp_no) == str(3) or str(wp_no) == str(4):
                                                    created_file,subDirectory,directory = CreateFolder(folderName,subFname,item,dataset) 
                                                    data_split_types(directory,subDirectory,created_file,wp_no,valve_no["number"])
                                            else:
                                                created_file, subDirectory,directory = CreateFolder(folderName,subFname,item,dataset)
                                                data_split_types(directory,subDirectory,created_file,wp_no,valve_no["number"])        

    return Integrated_folders;    
                            